#### Use selenium and beautifulsoup to crawl data from lazada

In [ ]:
# Install necesary packages
""" %pip install selenium 
%pip install beautifulsoup4
%pip install pandas
%pip install requests """

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


In [42]:
def product_on_page(url, keyword):
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium import webdriver
    import time

    browser = webdriver.Chrome()
    browser.maximize_window()
    browser.implicitly_wait(20)
    
    browser.get(url)
    search_box = browser.find_element(By.ID, "q")
    search_box.send_keys(keyword)
    search_box.submit()
    
    WebDriverWait(browser, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".Bm3ON"))
    )
    
    products = []
    current_page = 0
    
    while current_page < 10:
        print(f"Load {current_page + 1}...")
        product_elements = browser.find_elements(By.CSS_SELECTOR, ".Bm3ON")
        
        for product in product_elements:
            try:
                link_element = product.find_element(By.CSS_SELECTOR, "a")
                link = link_element.get_attribute("href")
                print(link)
                
                """ span_element = product.find_element(By.TAG_NAME, 'span')
                content = span_element.text
                print("Content tag span:", content)"""

                try:
                    price_element = product.find_element(By.CLASS_NAME, "ooOxS")
                    price = price_element.text
                except Exception:
                    price = None                  

                try:
                    sold_element = product.find_element(By.CLASS_NAME, "_1cEkb")
                    sold = sold_element.text
                except Exception:
                    sold = None   

                products.append({
                    "link": link,
                    "price": price,
                    "sold": sold
                }) 
            except Exception as e:
                print(f"Error fetching product details: {e}")
        
        try:
            next_button = browser.find_element(By.CSS_SELECTOR, "li.ant-pagination-next")
            if "ant-pagination-disabled" in next_button.get_attribute("class"):
                print("Not next page.")
                break
            browser.execute_script("arguments[0].click();", next_button)
            time.sleep(2)   
            current_page += 1
        except Exception as e:
            print("Can not continue:", e)
            break
    
    browser.quit()
    
    return products


In [43]:
url = 'https://www.lazada.vn/'
keyword = "iphone"
product_links = product_on_page(url, keyword)

Load 1...
https://www.lazada.vn/products/duy-nhat-26-2811-iphone-16-pro-max-hang-chinh-hang-vna-i2792189799.html
https://www.lazada.vn/products/apple-iphone-15-pro-max-256gb-chinh-hang-vna-i2416996609.html
https://www.lazada.vn/products/apple-iphone-15-plus-128gb-chinh-hang-vna-i2417162029.html
https://www.lazada.vn/products/duy-nhat-26-2811-iphone-15-pro-max-hang-chinh-hang-vna-i2417233670.html
https://www.lazada.vn/products/apple-iphone-13-128gb-chinh-hang-vna-i1848816542.html
https://www.lazada.vn/products/iphone-14-plus-hang-chinh-hang-vna-i2017749016.html
https://www.lazada.vn/products/tra-gop-0-apple-iphone-15-pro-max-256gb-chinh-hang-vna-viettel-store-i2416609251.html
https://www.lazada.vn/products/duy-nhat-26-2811-iphone-15-plus-hang-chinh-hang-vna-i2417254495.html
https://www.lazada.vn/products/duy-nhat-26-2811-iphone-13-hang-chinh-hang-vna-i1525577176.html
https://www.lazada.vn/products/tra-gop-0-dien-thoai-apple-iphone-11-chinh-hang-vna-viettel-store-i1453408210.html
https:/

In [45]:
print(len(product_links))

400
